In [ ]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Read Dataset
df = pd.read_parquet('../data/final_dataset_v3.parquet')
df.head()

In [ ]:
train_start_date = '2020-01-01'
train_end_date = '2021-11-30'

test_start_date = '2021-11-30'
test_end_date = '2022-11-01'

features_diff = ['Partially Vaccinated', 'Fully Vaccinated', 'Non Vaccinated', 'daily_cases',
                    'daily_deaths', 'Patients Age 65 to 74 Years', 'Patients Age 55 to 64 Years',
                     'Patients Currently Hospitalized', 'Patients Age 55 to 64 Years']

features = ['overload-14day', 'overload', 'day_of_week', 'day_of_month',
            'parks_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline']

# How many days back to get the difference
nshift=4

# Forecast period
forecast=14



def prepare_input_data(df, train_start_date, train_end_date, test_start_date, test_end_date,
                        features, features_diff, nshift=4, forecast=14):
    # subset date range
    subset = df.copy()
    subset['day_of_week'] = subset.index.get_level_values('date').dayofweek
    subset['day_of_month'] = subset.index.get_level_values('date').day
    subset['overload'] = subset['Number of Beds Available'] + \
        subset['Number of ICU Beds Available'] - subset['Patients Newly Admitted']

    # diff column with difference between yesterday and day
    diff_cols = []
    for i in range(1, nshift):
        subset.loc[:, 'overload_T-' +
                str(i)] = subset.groupby(level=0)['overload'].shift(i)
        subset.loc[:, 'overload_T-' +
                str(i) + '_diff'] = subset.groupby(level=0)['overload_T-'+str(i)].diff()
        diff_cols.append('overload_T-' +
                        str(i))
        diff_cols.append('overload_T-'+str(i) + '_diff')

    for col in features_diff:
        subset.loc[:, col+'_diff'] = subset.groupby(level=0)[col].diff()
        # diff_cols.append(col)
        diff_cols.append(col+'_diff')

    # overload days in future
    subset.loc[:, f'overload-{forecast}day'] = subset.groupby(level=0)['overload'].shift(forecast)
    # dropping NAs
    subset = subset.dropna()

    target = f'overload-{forecast}day'
    cols = features + diff_cols
    # cols = ['overload-14day', 'overload', *diff_cols, 'day_of_week', 'day_of_month',
    #         'parks_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline']

    subset = subset[cols]
    # subset = subset[(subset.index.get_level_values('date') >= '2021-11-01') &
    #                 (subset.index.get_level_values('date') <= '2022-01-31')]


    subset_test = subset[(subset.index.get_level_values('date') > test_start_date) & (subset.index.get_level_values('date') <= test_end_date)]
    # subset_test = subset[(subset.index.get_level_values('date') > test_start_date)]
    subset = subset[(subset.index.get_level_values('date') >= train_start_date) & (subset.index.get_level_values('date') <= train_end_date)]
    # subset = subset[(subset.index.get_level_values('date') <= train_end_date)]

    # subset.tail(10)


    # subset_train = subset[(subset.index.get_level_values('date') >= '2021-11-01') &
    #                       (subset.index.get_level_values('date') <= '2022-01-31')]
    # subset_test = subset[(subset.index.get_level_values('date') >= '2022-02-01') &
    #                      (subset.index.get_level_values('date') <= '2022-02-28')]
    # subset_train.shape, subset_test.shape

    return subset, subset_test